In [ ]:
#!pip install google-search-results

In [1]:
import openai
from dotenv import load_dotenv
import os
from serpapi import GoogleSearch
import json

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_api_key")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

client = openai.OpenAI()

In [2]:
params = {
  "engine": "google",
  "q": "coffee manufacturers",
  "api_key": os.environ["SERPAPI_API_KEY"]
}

search = GoogleSearch(params)
search.get_dict()['organic_results']



[{'position': 1,
  'title': 'Coffee Manufactory',
  'link': 'https://www.coffeemanufactory.com/',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.coffeemanufactory.com/&ved=2ahUKEwi7xcXPh-eJAxUOJNAFHeXGLg4QFnoECB4QAQ',
  'displayed_link': 'https://www.coffeemanufactory.com',
  'thumbnail': 'https://serpapi.com/searches/673bd0fd4d52b8ef2ea7830b/images/355a5be17f94f0c402907a9c741b3722fc3e49f38637129897e38505c2030b41.jpeg',
  'favicon': 'https://serpapi.com/searches/673bd0fd4d52b8ef2ea7830b/images/355a5be17f94f0c402907a9c741b37229645be00ff4a2c1704a55394aef271c8.png',
  'snippet': 'Coffee Manufactory brings better coffee to more people through quality, education, and immersion.',
  'snippet_highlighted_words': ['Coffee', 'coffee'],
  'source': 'Coffee Manufactory'},
 {'position': 2,
  'title': 'Arabica Coffee Roasters',
  'link': 'https://www.arabicacoffeeco.com/',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=899

In [3]:
def search_google_news(query):
    params = {
        "engine": "google_news",
        "time_period": "last_day",
        "q": query,
        "api_key": os.environ["SERPAPI_API_KEY"]
    }

    num_results = 5
    search = GoogleSearch(params)
    results = search.get_dict()['news_results']
    extracted_results = []
    for result in results[:num_results]:
        extracted_results.append({
            "title": result.get("title"),
            "link": result.get("link"),
            "snippet": result.get("snippet", "No snippet available")
        })
    
    return extracted_results



In [4]:
search_google_news("supreme court")

[{'title': 'US Supreme Court rejects challenge to Alaska campaign finance law',
  'link': 'https://www.reuters.com/legal/us-supreme-court-rejects-challenge-alaska-campaign-finance-law-2024-11-18/',
  'snippet': 'No snippet available'},
 {'title': 'Supreme Court won’t hear challenge to Alaska campaign finance laws',
  'link': 'https://www.scotusblog.com/2024/11/supreme-court-wont-hear-challenge-to-alaska-campaign-finance-laws/',
  'snippet': 'No snippet available'},
 {'title': 'Democrat keeps slight lead for North Carolina Supreme Court as counties finish count',
  'link': 'https://abcnews.go.com/Politics/wireStory/democrat-slight-lead-north-carolina-supreme-court-counties-115988625',
  'snippet': 'No snippet available'},
 {'title': 'Wisconsin supreme court weighs fate of top elections official targeted by right',
  'link': 'https://www.theguardian.com/us-news/2024/nov/18/wisconsin-meagan-wolfe-supreme-court',
  'snippet': 'No snippet available'},
 {'title': "Pa. Supreme Court says unda

In [5]:
parameters = {
    "type": "object",
    "properties": {
        "query": {
            "type": "string",
            "description": "search query"
        }
    },
    "required": ["query"]
}

search_google_news_tool = {
    "type": "function",
    "function": {
        "name": "search_google_news",
        "description": "Retreive Google news results from last 24 hours for a given query.",
        "parameters": parameters,
    }
}

In [6]:
def get_output(messages_input, model_input="gpt-4o-mini", temperature_input = 0) :

    response = client.chat.completions.create(
        model = model_input,
        messages = messages_input,
        temperature = temperature_input,
        tools = [search_google_news_tool],
        tool_choice = "auto"
    )
    
    response = response.choices[0].message
    return response

In [7]:
def collect_messages(prompt, context):
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_output(context) 
    context.append({'role':'assistant', 'content':f"{response.content}"})
    return response

In [14]:
def main():
    system_message = f"""
    You are a chatbot that can use Google News to help users locate recent news details.
    Greet the user and ask what they are interesting in searching.
    Use the Google results to return and the most current information.
    SUmmarize the results in a brief 2 sentence summary.
    """
    context = [{"role":"system", "content":system_message}]


    print("Welcome! Type 'exit' to end the chat.")
    greeting = collect_messages("", context)
    print(greeting.content)
    
    while True:
        user_input = input()
        
        if user_input.lower() == 'exit':
            return context
        
        response = collect_messages(user_input, context)

        if response.content:
            print(response.content)
        elif response.tool_calls:
            arguments = json.loads(response.tool_calls[0].function.arguments)
            query = arguments["query"]
            search_results = search_google_news(query)
            context.append({'role':'function', \
                            'name': 'search_google_news', \
                            'content':f"{search_results}"})
            response = collect_messages("summarize news for userin email", context)
            print(response.content)
    
 

In [ ]:
main()

Welcome! Type 'exit' to end the chat.
Hello! How can I assist you today? What news topic are you interested in searching for?


 spirit airlines


Spirit Airlines has recently filed for Chapter 11 bankruptcy, marking a significant development for the budget travel airline that has reportedly lost $2.2 billion over the past five years. This situation raises questions about the implications for travelers, as the airline navigates its financial challenges. 

For more details, you can check the full articles [here](https://www.msnbc.com/morning-joe/watch/budget-travel-airline-spirit-files-for-bankruptcy-224744517782), [here](https://www.nytimes.com/2024/11/18/business/spirit-airlines-bankruptcy.html), and [here](https://www.cbsnews.com/news/spirit-airlines-files-for-bankruptcy/).
